In [1]:
import sys
sys.path.append('../')

from rocket_rag.utils import *
from rocket_rag.agent import RagAgent

c:\Users\HAOXUAN\miniconda3\envs\agents2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load = '40kg'

In [3]:
if_files_dict = parse_files(main_directory=INFERENCE_DIR)
if_ts_files = if_files_dict[load]
rand_idx = np.random.randint(0, len(if_ts_files))
if_ts_filename = if_ts_files[rand_idx]
print(f'Randomly selected file: {if_ts_filename}')

if_rocket_feature = fit_transform(ts_filename=[if_ts_filename],
                                  field='current',
                                  smooth=True,
                                  smooth_ws=15,
                                  tolist=False,
                                  verbo=True)

2024-05-29 17:12:06.492 | DEBUG    | rocket_rag.utils:fit_transform:149 - Extract the time series data points
2024-05-29 17:12:06.502 | INFO     | rocket_rag.utils:fit:125 - time series extracted SUCCESSFULLY.
2024-05-29 17:12:06.503 | DEBUG    | rocket_rag.utils:fit_transform:161 - Rocket transforming...


Randomly selected file: ../data/inference/40kg\spalling7\spalling7_40_3_3.csv


In [4]:
agent = RagAgent(vs_dir=f'../store/nodes_{load}.pkl')

2024-05-29 17:12:10.745 | DEBUG    | rocket_rag.node_indexing:load_node_indexing:98 - Loading all nodes...
2024-05-29 17:12:11.135 | INFO     | rocket_rag.node_indexing:load_node_indexing:102 - All nodes are loaded.
2024-05-29 17:12:11.467 | INFO     | rocket_rag.agent:_init_openai_client:67 - OpenAI API key found. OpenAI client Initialized.
2024-05-29 17:12:11.468 | INFO     | rocket_rag.agent:__init__:59 - RAG agent initialized.


In [5]:
rocket_feature = agent.get_rocket_feature(if_ts_filename)
print(f'Rocket feature shape: {rocket_feature.shape}')
fault_prediction, score = agent.get_fault_prediction()
print(f'Fault prediction: {fault_prediction}, Score: {score}')

2024-05-29 17:12:11.474 | DEBUG    | rocket_rag.utils:fit_transform:149 - Extract the time series data points
2024-05-29 17:12:11.477 | INFO     | rocket_rag.utils:fit:125 - time series extracted SUCCESSFULLY.
2024-05-29 17:12:11.478 | DEBUG    | rocket_rag.utils:fit_transform:161 - Rocket transforming...


Rocket feature shape: (1, 20000)
Fault prediction: ['spalling7_40_4_4'], Score: 1.0


In [6]:
agent.generate_fault_diagnosis_statement()
print(agent.fault_diagnosis_json)

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
agent.refine_fault_diagnosis_statement()
print(agent.refined_diagnosis_json)

{'fault_type': 'spalling', 'degradation_level': 1, 'refinement': 'No obvious fault detection'}


In [ ]:
from prompts import multi_queries_gen_prompt

fault_type = agent.refined_diagnosis_json['refinement']
fault_description = agent.fault_diagnosis_json['description']
sys_prompt = multi_queries_gen_prompt.sys_prompt
user_prompt = multi_queries_gen_prompt.user_prompt.format(fault_type=fault_type, 
                                                          fault_description=fault_description,
                                                          num=str(5))
messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": user_prompt}
]

agent.generate_response(prompts=messages)

ChatCompletion(id='chatcmpl-9UGA6ADB1L5fy8yazlpCq6lGHXONL', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_vIJSAAXxfLNaJhRJSMyYppSK', function=Function(arguments='{"file_name":"linear_actuators_fault_queries.txt","contents":"how to repair ball-screw mechanism surface damage on linear actuators\\nhow to improve smoothness and efficiency of linear actuators with ball-screw mechanism surface damage\\nmaintenance methods for ball-screw mechanism surface damage on linear actuators\\nrecovery techniques for surface damage in ball-screw mechanism of linear actuators\\nhow to address low-level surface damage in ball-screw mechanism of linear actuators"}', name='save_to_file'), type='function')]))], created=1716998994, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=104, pr